In [40]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader

In [41]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])


In [42]:
dataset = ImageFolder(root='./dataTrain', transform=transform)

In [43]:
from torch.utils.data import random_split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [44]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [45]:
class_name = dataset.classes
print("Classes:", class_name)

Classes: ['Backpack', 'Crossbody Bag', 'Handbag', 'Shoulder Bag', 'Tote Bag', 'clutch bag']


In [46]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 12, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(12, 24, 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(class_name))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        zx = F.relu(self.fc1(x))
        x = F.relu(self.fc2(zx))
        x = self.fc3(x)
        return x

In [47]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [48]:
for epoch in range(30):
    print(f'Traning epoch {epoch}...')
    
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        print(torch.unique(labels)) 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {running_loss:.3f}")

Traning epoch 0...
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
tensor([0, 1, 2, 3, 4, 5])
Epoch 1 loss: 60.412
Traning epoch 1

In [49]:
torch.save(net.state_dict(), 'trained_model.pth')

In [50]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_model.pth'))
net.eval()

NeuralNet(
  (conv1): Conv2d(3, 12, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(12, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=600, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=6, bias=True)
)

In [51]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy}%')

Accuracy: 50.36231884057971%


In [52]:
from PIL import Image
import torch
import torchvision.transforms as transforms

# Định nghĩa các phép biến đổi
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Hàm load hình ảnh
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = new_transform(image)
    image = image.unsqueeze(0)  # Thêm chiều batch vào (nếu cần)
    return image

# Đường dẫn hình ảnh (lưu ý rằng bạn cần tách các đường dẫn ra thành các phần tử trong danh sách)
image_paths = ['test1.jpg', 'test2.jpg']  # Sửa thành danh sách chứa các đường dẫn riêng biệt
images = [load_image(img) for img in image_paths]

# Đảm bảo mạng ở chế độ evaluation
net.eval()

# Dự đoán cho các hình ảnh
with torch.no_grad():
    for image in images:
        outputs = net(image)
        _, predicted = torch.max(outputs.data, 1)
        print(f'Predicted class: {class_name[predicted.item()]}')


Predicted class: Tote Bag
Predicted class: clutch bag
